In [9]:
!pip3 install datasets transformers -q

In [10]:
!pip3 install wandb --upgrade -q

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from datasets import load_dataset
import random
from sklearn import metrics, model_selection, preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup

In [12]:
def seed_everything(seed=73):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # some cudnn methods can be random even after fixing the seed unless you tell it to be deterministic
    torch.backends.cudnn.deterministic = True

seed_everything(1234)

In [13]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [14]:
sweep_config = {
    'method': 'random', #grid, random, bayesian
    'metric': {
      'name': 'f1_micro',
      'goal': 'maximize'   
    },
    'parameters': {

        'learning_rate': {
            'values': [3e-5]
        },
        'batch_size': {
            'values': [64]
        },
        'epochs':{'value': 10},
        'dropout':{
            'values': [0.3]
        },
        'tokenizer_max_len': {'value': 40},
    }
}

sweep_defaults = {
    'learning_rate': 3e-5,
    'batch_size': 64,
    'epochs': 10,
    'dropout': 0.3,
    'tokenizer_max_len': 40
}

sweep_id = wandb.sweep(sweep_config, project='SqueezeBERT_Classification')

Create sweep with ID: r1yww5eq
Sweep URL: https://wandb.ai/kmaas/SqueezeBERT_Classification/sweeps/r1yww5eq


In [15]:
# go_emotions = load_dataset("go_emotions")
# data = go_emotions.data
# Mounting drive and importing csv's
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/MyDrive/Applied_ML_Project/')
TRAIN_PATH = '/content/drive/MyDrive/Applied_ML_Project/data/train_cleaned.csv'
TEST_PATH = '/content/drive/MyDrive/Applied_ML_Project/data/test_cleaned.csv'
VAL_PATH = '/content/drive/MyDrive/Applied_ML_Project/data/val_cleaned.csv'
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
valid = pd.read_csv(VAL_PATH)

Mounted at /content/drive


In [16]:
print(train.shape, test.shape, valid.shape)

(18555, 14) (2320, 14) (2319, 14)


In [17]:
train.dropna(inplace = True)

In [18]:
train.head()

,Unnamed: 0,index,Policy_text,Category_Data Retention,Category_Data Security,Category_Do Not Track,Category_First Party Collection/Use,Category_International and Specific Audiences,Category_Other,Category_Policy Change,Category_Third Party Sharing/Collection,"Category_User Access, Edit and Deletion",Category_User Choice/Control,text
0,8395,8395,<strong> Changing/deleting personal informatio...,0,0,0,0,0,1,0,0,0,0,changing deleting personal information if y...
1,15718,15718,Kaleida Health will not be required to obtain ...,0,0,0,0,0,0,0,0,0,1,kaleida health will not be required to obtain ...
2,14983,14983,Law enforcement officials and as required by l...,0,0,0,0,0,0,0,1,0,0,law enforcement officials and as required by l...
3,6271,6271,Use of Cookies Similar Tracking Technologies <...,0,0,0,0,0,0,0,0,0,1,use of cookies similar tracking technologies ...
4,14040,14040,"Agents: We employ other businesses, certain se...",0,0,0,0,0,0,0,1,0,0,agents we employ other businesses certain se...


In [19]:
# Restructring Datasets
train_labels = np.argmax(train.iloc[:,  3:-2].values, axis=1)
test_labels = np.argmax(test.iloc[:, 3:-2].values, axis=1)
val_labels = np.argmax(valid.iloc[:, 3:-2].values, axis=1)

train['labels'] = train_labels
test['labels'] = test_labels
valid['labels'] = val_labels

train.head()

,Unnamed: 0,index,Policy_text,Category_Data Retention,Category_Data Security,Category_Do Not Track,Category_First Party Collection/Use,Category_International and Specific Audiences,Category_Other,Category_Policy Change,Category_Third Party Sharing/Collection,"Category_User Access, Edit and Deletion",Category_User Choice/Control,text,labels
0,8395,8395,<strong> Changing/deleting personal informatio...,0,0,0,0,0,1,0,0,0,0,changing deleting personal information if y...,5
1,15718,15718,Kaleida Health will not be required to obtain ...,0,0,0,0,0,0,0,0,0,1,kaleida health will not be required to obtain ...,0
2,14983,14983,Law enforcement officials and as required by l...,0,0,0,0,0,0,0,1,0,0,law enforcement officials and as required by l...,7
3,6271,6271,Use of Cookies Similar Tracking Technologies <...,0,0,0,0,0,0,0,0,0,1,use of cookies similar tracking technologies ...,0
4,14040,14040,"Agents: We employ other businesses, certain se...",0,0,0,0,0,0,0,1,0,0,agents we employ other businesses certain se...,7


In [20]:
# train, valid, test = data["train"].to_pandas(), data["validation"].to_pandas(), data["test"].to_pandas()

In [21]:
print(train.shape, valid.shape, test.shape) 

(18552, 15) (2319, 15) (2320, 15)


In [22]:
train.head()

,Unnamed: 0,index,Policy_text,Category_Data Retention,Category_Data Security,Category_Do Not Track,Category_First Party Collection/Use,Category_International and Specific Audiences,Category_Other,Category_Policy Change,Category_Third Party Sharing/Collection,"Category_User Access, Edit and Deletion",Category_User Choice/Control,text,labels
0,8395,8395,<strong> Changing/deleting personal informatio...,0,0,0,0,0,1,0,0,0,0,changing deleting personal information if y...,5
1,15718,15718,Kaleida Health will not be required to obtain ...,0,0,0,0,0,0,0,0,0,1,kaleida health will not be required to obtain ...,0
2,14983,14983,Law enforcement officials and as required by l...,0,0,0,0,0,0,0,1,0,0,law enforcement officials and as required by l...,7
3,6271,6271,Use of Cookies Similar Tracking Technologies <...,0,0,0,0,0,0,0,0,0,1,use of cookies similar tracking technologies ...,0
4,14040,14040,"Agents: We employ other businesses, certain se...",0,0,0,0,0,0,0,1,0,0,agents we employ other businesses certain se...,7


In [23]:
for col in train.columns:
  print(col)

Unnamed: 0
index
Policy_text
Category_Data Retention
Category_Data Security
Category_Do Not Track
Category_First Party Collection/Use
Category_International and Specific Audiences
Category_Other
Category_Policy Change
Category_Third Party Sharing/Collection
Category_User Access, Edit and Deletion
Category_User Choice/Control
text
labels


In [24]:
mapping = {
    0:"Category_Data Retention",
    1:"Category_Data Security",
    2:"Category_Do Not Track",
    3:"Category_First Party Collection/Use",
    4:"Category_International and Specific Audiences",
    5:"Category_Other",
    6:"Category_Policy Change",
    7:"Category_Third Party Sharing/Collection",
    8:"Category_User Access, Edit and Deletion",
    9:"Category_User Choice/Control",
}

n_labels = len(mapping)

In [25]:
# def one_hot_encoder(df):
#     one_hot_encoding = []
#     for i in tqdm(range(len(df))):
#         temp = [0]*n_labels
#         label_indices = df.iloc[i]["labels"]
#         for index in label_indices:
#             temp[index] = 1
#         one_hot_encoding.append(temp)
#     return pd.DataFrame(one_hot_encoding)

In [26]:
# train_ohe_labels = one_hot_encoder(train)
# valid_ohe_labels = one_hot_encoder(valid)
# test_ohe_labels = one_hot_encoder(test)

In [27]:
# train_ohe_labels.shape

In [28]:
# train = pd.concat([train, train_ohe_labels], axis=1)
# valid = pd.concat([valid, valid_ohe_labels], axis=1)
# test = pd.concat([test, test_ohe_labels], axis=1)

In [29]:
train['labels'] = train['labels'].apply(lambda x: np.array([x]))
valid['labels'] = valid['labels'].apply(lambda x: np.array([x]))
test['labels'] = test['labels'].apply(lambda x: np.array([x]))


In [30]:
train

,Unnamed: 0,index,Policy_text,Category_Data Retention,Category_Data Security,Category_Do Not Track,Category_First Party Collection/Use,Category_International and Specific Audiences,Category_Other,Category_Policy Change,Category_Third Party Sharing/Collection,"Category_User Access, Edit and Deletion",Category_User Choice/Control,text,labels
0,8395,8395,<strong> Changing/deleting personal informatio...,0,0,0,0,0,1,0,0,0,0,changing deleting personal information if y...,[5]
1,15718,15718,Kaleida Health will not be required to obtain ...,0,0,0,0,0,0,0,0,0,1,kaleida health will not be required to obtain ...,[0]
2,14983,14983,Law enforcement officials and as required by l...,0,0,0,0,0,0,0,1,0,0,law enforcement officials and as required by l...,[7]
3,6271,6271,Use of Cookies Similar Tracking Technologies <...,0,0,0,0,0,0,0,0,0,1,use of cookies similar tracking technologies ...,[0]
4,14040,14040,"Agents: We employ other businesses, certain se...",0,0,0,0,0,0,0,1,0,0,agents we employ other businesses certain se...,[7]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18550,11964,11964,<strong> WITH WHOM DO WE SHARE THE INFORMATION...,0,0,0,0,0,0,0,1,0,0,with whom do we share the information that we ...,[7]
18551,21575,21575,We may also file your comments to improve the ...,0,0,0,1,0,0,0,0,0,0,we may also file your comments to improve the ...,[3]
18552,5390,5390,Our servers may also automatically collect inf...,0,0,0,1,0,0,0,0,0,0,our servers may also automatically collect inf...,[3]
18553,860,860,My Favorites. You may choose to register for a...,0,0,0,1,0,0,0,0,0,0,my favorites you may choose to register for a...,[3]


In [31]:
train = train.rename(columns={'index': 'id'})
train.drop(['Unnamed: 0', 'Policy_text'], axis=1, inplace = True)

valid = valid.rename(columns={'index': 'id'})
valid.drop(['Unnamed: 0', 'Policy_text'], axis=1, inplace = True)

test = test.rename(columns={'index': 'id'})
test.drop(['Unnamed: 0', 'Policy_text'], axis=1, inplace = True)
train.head()

,id,Category_Data Retention,Category_Data Security,Category_Do Not Track,Category_First Party Collection/Use,Category_International and Specific Audiences,Category_Other,Category_Policy Change,Category_Third Party Sharing/Collection,"Category_User Access, Edit and Deletion",Category_User Choice/Control,text,labels
0,8395,0,0,0,0,0,1,0,0,0,0,changing deleting personal information if y...,[5]
1,15718,0,0,0,0,0,0,0,0,0,1,kaleida health will not be required to obtain ...,[0]
2,14983,0,0,0,0,0,0,0,1,0,0,law enforcement officials and as required by l...,[7]
3,6271,0,0,0,0,0,0,0,0,0,1,use of cookies similar tracking technologies ...,[0]
4,14040,0,0,0,0,0,0,0,1,0,0,agents we employ other businesses certain se...,[7]


In [32]:
train = train.rename(columns = {"Category_Data Retention": 0,
                              "Category_Data Security":1, 
                              "Category_Do Not Track":2,
                              "Category_First Party Collection/Use":3,
                              "Category_International and Specific Audiences":4,
                              "Category_Other":5,
                              "Category_Policy Change":6,
                              "Category_Third Party Sharing/Collection":7,
                              "Category_User Access, Edit and Deletion":8,
                              "Category_User Choice/Control":9,
                              })
test = test.rename(columns = {"Category_Data Retention": 0,
                              "Category_Data Security":1, 
                              "Category_Do Not Track":2,
                              "Category_First Party Collection/Use":3,
                              "Category_International and Specific Audiences":4,
                              "Category_Other":5,
                              "Category_Policy Change":6,
                              "Category_Third Party Sharing/Collection":7,
                              "Category_User Access, Edit and Deletion":8,
                              "Category_User Choice/Control":9,
                              })
valid = valid.rename(columns={"Category_Data Retention": 0,
                              "Category_Data Security":1, 
                              "Category_Do Not Track":2,
                              "Category_First Party Collection/Use":3,
                              "Category_International and Specific Audiences":4,
                              "Category_Other":5,
                              "Category_Policy Change":6,
                              "Category_Third Party Sharing/Collection":7,
                              "Category_User Access, Edit and Deletion":8,
                              "Category_User Choice/Control":9,
                              })


In [33]:
train.head()

,id,0,1,2,3,4,5,6,7,8,9,text,labels
0,8395,0,0,0,0,0,1,0,0,0,0,changing deleting personal information if y...,[5]
1,15718,0,0,0,0,0,0,0,0,0,1,kaleida health will not be required to obtain ...,[0]
2,14983,0,0,0,0,0,0,0,1,0,0,law enforcement officials and as required by l...,[7]
3,6271,0,0,0,0,0,0,0,0,0,1,use of cookies similar tracking technologies ...,[0]
4,14040,0,0,0,0,0,0,0,1,0,0,agents we employ other businesses certain se...,[7]


In [34]:
train['labels'][1]

array([0])

In [35]:
def inspect_category_wise_data(label, n=5):
    samples = train[train[label] == 1].sample(n)
    sentiment = mapping[label]
    
    print(f"{n} samples from {sentiment} sentiment: \n")
    for text in samples["text"]:
        print(text, end='\n\n')

In [36]:
inspect_category_wise_data(4)

5 samples from Category_International and Specific Audiences sentiment: 

we will retain your or your childs information for as long as your account is active or as needed to provide you services  if you wish to cancel your or your childs account or request that we no longer use your information to provide you services contact us at support lynda com  we will retain and use your information as necessary to comply with our legal obligations  resolve disputes  and enforce our agreements  we will respond to your request to access within 30 days

for additional tips on how to help children stay safe on the internet  we recommend that you visit the following sites    www ftc gov privacy privacyinitiatives childrens html   www ftc gov bcp edu pubs consumer tech tec14 shtm   http   www onguardonline gov features feature 0002 featured info parents

what is our policy on children s privacy     the sites are not for children under 13 years of age  however  there are some areas of the sites that 

In [37]:
class GoEmotionDataset:
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels

        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]

        inputs = self.tokenizer.__call__(text,
                                        None,
                                        add_special_tokens=True,
                                        max_length=self.max_len,
                                        padding="max_length",
                                        truncation=True,
                                        )
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "labels": torch.tensor(label, dtype=torch.long)
        }

In [38]:
class GoEmotionClassifier(nn.Module):
    def __init__(self, n_train_steps, n_classes, do_prob, bert_model):
        super(GoEmotionClassifier, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(do_prob)
        self.out = nn.Linear(768, n_classes)
        self.n_train_steps = n_train_steps
        self.step_scheduler_after = "batch"

    def forward(self, ids, mask):
        output_1 = self.bert(ids, attention_mask=mask)["pooler_output"]
        output_2 = self.dropout(output_1)
        output = self.out(output_2)
        return output

In [39]:
tokenizer = transformers.SqueezeBertTokenizer.from_pretrained("squeezebert/squeezebert-uncased", do_lower_case=True)

def build_dataset(tokenizer_max_len):
    train_dataset = GoEmotionDataset(train.text.tolist(), train[range(n_labels)].values.tolist(), tokenizer, tokenizer_max_len)
    valid_dataset = GoEmotionDataset(valid.text.tolist(), valid[range(n_labels)].values.tolist(), tokenizer, tokenizer_max_len)
    
    return train_dataset, valid_dataset

def build_dataloader(train_dataset, valid_dataset, batch_size):
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    valid_data_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

    return train_data_loader, valid_data_loader

def ret_model(n_train_steps, do_prob):
  model = GoEmotionClassifier(n_train_steps, n_labels, do_prob, bert_model=bert_model)
  return model

In [40]:
sample_train_dataset, _ = build_dataset(40) #40 is the tokenizer length
print(sample_train_dataset[0])
len(sample_train_dataset)

{'ids': tensor([  101,  5278,  3972, 20624,  3070,  3167,  2592,  2065,  2017,  2052,
         2066,  2000,  2031,  2115,  7714, 27800,  2592,  2904,  2030,  3718,
         2013,  2256,  7809,  3531,  3967,  2149,  2012, 18558,  2088, 19779,
         4012,  1998,  2057,  2097,  2224,  9608,  4073,  2000, 14037,   102]), 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), 'labels': tensor([0, 0, 0, 0, 0, 1, 0, 0, 0, 0])}


18552

In [41]:
bert_model = transformers.SqueezeBertModel.from_pretrained("squeezebert/squeezebert-uncased")

Some weights of the model checkpoint at squeezebert/squeezebert-uncased were not used when initializing SqueezeBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing SqueezeBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SqueezeBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [42]:
def ret_optimizer(model):
    '''
    Taken from Abhishek Thakur's Tez library example: 
    https://github.com/abhishekkrthakur/tez/blob/main/examples/text_classification/binary.py
    '''
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]
    opt = AdamW(optimizer_parameters, lr=wandb.config.learning_rate)
    return opt

def ret_scheduler(optimizer, num_train_steps):
    sch = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
    return sch

def loss_fn(outputs, labels):
    if labels is None:
        return None
    return nn.BCEWithLogitsLoss()(outputs, labels.float())

def log_metrics(preds, labels):
    preds = torch.stack(preds)
    preds = preds.cpu().detach().numpy()
    preds1 = np.argmax(preds, axis=1)
    labels = torch.stack(labels)
    labels = labels.cpu().detach().numpy()
    labels1 = np.argmax(labels, axis=1)
    print("Preds:",preds)
    print("Labels:",labels)
    
    '''
    auc_micro_list = []
    for i in range(n_labels):
      current_pred = preds.T[i]
      current_label = labels.T[i]
      fpr_micro, tpr_micro, _ = metrics.roc_curve(current_label.T, current_pred.T)
      auc_micro = metrics.auc(fpr_micro, tpr_micro)
      auc_micro_list.append(auc_micro)
    
    return {"auc": np.array(auc_micro).mean()}
    '''
    # auc_micro_list = []
    # for i in range(n_labels):
    #   current_pred = preds.T[i]
    #   current_label = labels.T[i]
    #   fpr_micro, tpr_micro, _ = metrics.roc_curve(current_label.T, current_pred.T)
    #   auc_micro = metrics.auc(fpr_micro, tpr_micro)
    #   auc_micro_list.append(auc_micro)
    
    # return {"Micro AUC": np.array(auc_micro).mean()}
    
    #AUC Micro

    fpr_micro, tpr_micro, _ = metrics.roc_curve(labels.ravel(), preds.ravel())
    
    auc_micro = metrics.auc(fpr_micro, tpr_micro)
    # #F1 Score

    # calculate micro F1 score
    micro_f1 = f1_score(labels1, preds1, average='micro')
    # print("micro f1",micro_f1)
    macro_f1 = f1_score(labels1, preds1, average='macro')
    # print("macro f1",macro_f1)
    # calculate other metrics
    accuracy = accuracy_score(labels1, preds1)
    # print("accuracy",accuracy)
    # precision = precision_score(labels, preds, average='macro')
    recall = recall_score(labels1, preds1, average='macro')
    # print("recall",recall)
    # auc_micro = roc_auc_score(labels, preds, average='micro', multi_class='ovr')
    # print("auc micro",auc_micro)
    # auc_macro = roc_auc_score(labels, preds, average='macro', multi_class='ovr')
    # print("auc macro",auc_macro)
    
    return {
        "accuracy": accuracy,
        "recall": recall,
        "auc_micro": auc_micro,
        "micro_f1": micro_f1,
        "macro_f1": macro_f1
    }

In [43]:
def train_fn(data_loader, model, optimizer, device, scheduler):
    '''
        Modified from Abhishek Thakur's BERT example: 
        https://github.com/abhishekkrthakur/bert-sentiment/blob/master/src/engine.py
    '''

    train_loss = 0.0
    model.train()
    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d["ids"]
        mask = d["mask"]
        targets = d["labels"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask)

        loss = loss_fn(outputs, targets)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        scheduler.step()
    return train_loss
    

def eval_fn(data_loader, model, device):
    '''
        Modified from Abhishek Thakur's BERT example: 
        https://github.com/abhishekkrthakur/bert-sentiment/blob/master/src/engine.py
    '''
    eval_loss = 0.0
    model.eval()
    fin_targets = []
    fin_outputs = []
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            mask = d["mask"]
            targets = d["labels"]

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(ids=ids, mask=mask)
            loss = loss_fn(outputs, targets)
            eval_loss += loss.item()
            fin_targets.extend(targets)
            fin_outputs.extend(torch.sigmoid(outputs))
    return eval_loss, fin_outputs, fin_targets

In [44]:
def trainer(config=None):
    with wandb.init(config=config):
        config = wandb.config

        train_dataset, valid_dataset = build_dataset(config.tokenizer_max_len)
        train_data_loader, valid_data_loader = build_dataloader(train_dataset, valid_dataset, config.batch_size)
        print("Length of Train Dataloader: ", len(train_data_loader))
        print("Length of Valid Dataloader: ", len(valid_data_loader))
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        n_train_steps = int(len(train_dataset) / config.batch_size * 10)

        model = ret_model(n_train_steps, config.dropout)
        optimizer = ret_optimizer(model)
        scheduler = ret_scheduler(optimizer, n_train_steps)
        model.to(device)
        model = nn.DataParallel(model)
        wandb.watch(model)
        
        n_epochs = config.epochs

        best_val_loss = 100
        for epoch in tqdm(range(n_epochs)):
            train_loss = train_fn(train_data_loader, model, optimizer, device, scheduler)
            eval_loss, preds, labels = eval_fn(valid_data_loader, model, device)
          
            # auc_score = log_metrics(preds, labels)["auc_micro"]
            metrics = log_metrics(preds, labels)
            print("Accuracy:",metrics['accuracy'])
            print("Micro F1-Score:",metrics['micro_f1'])
            print("Macro F1-Score:",metrics['macro_f1'])
            print("Micro AUC score: ", metrics['auc_micro'])
            avg_train_loss, avg_val_loss = train_loss / len(train_data_loader), eval_loss / len(valid_data_loader)
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": avg_train_loss,
                "val_loss": avg_val_loss,
                "accuracy": metrics['accuracy'],
                "auc_micro_score": metrics['auc_micro'],
                "micro_f1": metrics['micro_f1'],
                "macro_f1": metrics['macro_f1']
            })

            print("Executing Epoch ",epoch)
            print("Average Train loss: ", avg_train_loss)
            print("Average Valid loss: ", avg_val_loss)
            print("Accuracy:",metrics['accuracy'])
            print("Micro F1-Score:",metrics['micro_f1'])
            print("Macro F1-Score:",metrics['macro_f1'])
            print("Micro AUC score: ", metrics['auc_micro'])
            curr_f1_micro = metrics['micro_f1']
            best_f1_micro = 0
            if best_f1_micro < curr_f1_micro:
                best_f1_micro = curr_f1_micro
                torch.save(model.state_dict(), "./Best_squeezebert_model.pt")  
                print("Model saved as current best F1-Micro is: ", best_f1_micro)
            else:
                print("Model not saved as current F1-Score is ",curr_f1_micro," which is lesser than the Best F1-Score: ",best_f1_micro)


            old_name = r"/content/Best_squeezebert_model.pt"
            new_name = r"/content/Bert_F1_"+str(round(best_f1_micro,2))+".pt"
            os.rename(old_name, new_name)


            import shutil
            shutil.copy("/content/Bert_F1_"+str(round(best_f1_micro,2))+".pt", "/content/drive/MyDrive/Applied_ML_Project/Classification Final Models/Bert_F1_"+str(round(best_f1_micro,2))+".pt")
            print("Copied Bert_F1_"+str(round(best_f1_micro,2))+".pt to Applied_ML_Project/Classification Final Models")

In [45]:
wandb.agent(sweep_id, function=trainer, count=6)

wandb: Agent Starting Run: mwz4hx35 with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.3
wandb: 	epochs: 10
wandb: 	learning_rate: 3e-05
wandb: 	tokenizer_max_len: 40
wandb: Currently logged in as: anishshetty-as (kmaas). Use `wandb login --relogin` to force relogin


Length of Train Dataloader:  290
Length of Valid Dataloader:  37


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/290 [00:00<?, ?it/s]

...

Preds: [[0.01251123 0.01261048 0.01307773 ... 0.03593774 0.01663203 0.01938508]
 [0.00364735 0.00383001 0.00420092 ... 0.01078109 0.00468418 0.01333728]
 [0.00252084 0.00305989 0.00194908 ... 0.00577586 0.00220976 0.00453154]
 ...
 [0.00324189 0.01575243 0.00117765 ... 0.68824387 0.00148803 0.00305859]
 [0.00740814 0.00718998 0.0074864  ... 0.8330679  0.00993013 0.09164899]
 [0.01952642 0.02398004 0.01585727 ... 0.03051657 0.01230603 0.02999176]]
Labels: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Accuracy: 0.7464424320827943
Micro F1-Score: 0.7464424320827943
Macro F1-Score: 0.6578038339706638
Micro AUC score:  0.9630964137925306
Executing Epoch  8
Average Train loss:  0.08915704852291222
Average Valid loss:  0.12743756154904495
Accuracy: 0.7464424320827943
Micro F1-Score: 0.7464424320827943
Macro F1-Score: 0.6578038339706638
Micro AUC score:  0.9630964137925306
Model saved 

  0%|          | 0/290 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

Preds: [[0.00287608 0.00419276 0.00106854 ... 0.23119764 0.0011758  0.00282146]
 [0.00549378 0.04651953 0.00166608 ... 0.43429524 0.00149549 0.00111953]
 [0.00656397 0.00237854 0.00132415 ... 0.00501373 0.00202248 0.00331987]
 ...
 [0.00409867 0.00603917 0.00111266 ... 0.00422264 0.00203256 0.00179754]
 [0.00856569 0.01331741 0.00689082 ... 0.01694304 0.0076156  0.00945802]
 [0.00208041 0.0034644  0.00084789 ... 0.42658743 0.00111778 0.00216414]]
Labels: [[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
Accuracy: 0.7485985338507978
Micro F1-Score: 0.7485985338507978
Macro F1-Score: 0.6611579677555512
Micro AUC score:  0.9632305774342396
Executing Epoch  9
Average Train loss:  0.08829222636233117
Average Valid loss:  0.12796629401477608
Accuracy: 0.7485985338507978
Micro F1-Score: 0.7485985338507978
Macro F1-Score: 0.6611579677555512
Micro AUC score:  0.9632305774342396
Model saved 

accuracy,▆██▁█▃▅▆▂▃
auc_micro_score,▁▄▆█▆██▇▇▇
epoch,▁▂▃▃▄▅▆▆▇█
macro_f1,▁▄▅▁█▂▆▇▆▇
micro_f1,▆██▁█▃▅▆▂▃
train_loss,█▃▂▂▁▁▁▁▁▁
val_loss,█▃▂▁▂▁▂▁▂▂
accuracy,0.7486
auc_micro_score,0.96323
epoch,10
macro_f1,0.66116


In [46]:
!zip -r ./SqueezeBertResults.zip ./wandb/

  adding: wandb/ (stored 0%)
  adding: wandb/run-20230424_003032-k2xlibyu/ (stored 0%)
  adding: wandb/run-20230424_003032-k2xlibyu/files/ (stored 0%)
  adding: wandb/run-20230424_003032-k2xlibyu/files/output.log (deflated 73%)
  adding: wandb/run-20230424_003032-k2xlibyu/files/wandb-metadata.json (deflated 56%)
  adding: wandb/run-20230424_003032-k2xlibyu/files/config.yaml (deflated 64%)
  adding: wandb/run-20230424_003032-k2xlibyu/files/requirements.txt (deflated 55%)
  adding: wandb/run-20230424_003032-k2xlibyu/files/wandb-summary.json (deflated 63%)
  adding: wandb/run-20230424_003032-k2xlibyu/logs/ (stored 0%)
  adding: wandb/run-20230424_003032-k2xlibyu/logs/debug.log (deflated 73%)
  adding: wandb/run-20230424_003032-k2xlibyu/logs/debug-internal.log (deflated 95%)
  adding: wandb/run-20230424_003032-k2xlibyu/tmp/ (stored 0%)
  adding: wandb/run-20230424_003032-k2xlibyu/tmp/code/ (stored 0%)
  adding: wandb/run-20230424_003032-k2xlibyu/run-k2xlibyu.wandb (deflated 63%)
  adding: 